In [ ]:
import pandas as pd

In [ ]:
file_path = '/content/accepted_2007_to_2018Q4 2.csv'
print("Veri seti okunuyor, lütfen bekleyin...")
df = pd.read_csv(file_path, low_memory=False)
print("veri seti okundu")

Veri seti okunuyor, lütfen bekleyin...
veri seti okundu


In [ ]:
loan_base = [
    'id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
    'installment', 'grade', 'sub_grade', 'purpose', 'title', 'issue_d',
    'loan_status', 'pymnt_plan', 'url', 'desc', 'initial_list_status',
    'disbursement_method', 'application_type', 'policy_code', ]

In [ ]:
borrower_info = [
    'id','emp_title', 'emp_length', 'home_ownership', 'annual_inc',
    'verification_status', 'addr_state', 'zip_code',
    'annual_inc_joint', 'verification_status_joint', 'dti', 'dti_joint', ]

In [ ]:
valid_loan_cols = [c for c in loan_base if c in df.columns]
valid_borrower_cols = [c for c in borrower_info if c in df.columns]

print(f"Loan Base için {len(valid_loan_cols)} sütun seçildi.")
print(f"Borrower Info için {len(valid_borrower_cols)} sütun seçildi.")

print("loan_base.csv oluşturuluyor...")
df[valid_loan_cols].to_csv('loan_base.csv', index=False)

print("borrower_info.csv oluşturuluyor...")
df[valid_borrower_cols].to_csv('borrower_info.csv', index=False)


Loan Base için 20 sütun seçildi.
Borrower Info için 12 sütun seçildi.
loan_base.csv oluşturuluyor...
borrower_info.csv oluşturuluyor...
✅ İŞLEM TAMAM! Dosyalar hazır.


In [ ]:
from google.colab import files

print("loan_base.csv indiriliyor...")
files.download('loan_base.csv')

print("borrower_info.csv indiriliyor...")
files.download('borrower_info.csv')

loan_base.csv indiriliyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

borrower_info.csv indiriliyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

# 1. Dosyaları Yükleme
df_loan = pd.read_csv('loan_base.csv', low_memory=False)
df_borrower = pd.read_csv('borrower_info.csv', low_memory=False)

# 1. Gereksiz Sütunları Silme (URL ve DESC)
cols_to_drop = ['url', 'desc']
df_loan.drop(columns=[c for c in cols_to_drop if c in df_loan.columns], inplace=True)

# 2. Initial List Status Açıklaması (W -> Whole, F -> Fractional)
status_map = {'w': 'Whole', 'f': 'Fractional'}
df_loan['initial_list_status'] = df_loan['initial_list_status'].map(status_map)

# 3. Title Sütunu Temizliği (Boşlukları NaN yapma)
df_loan['title'] = df_loan['title'].astype(str).str.strip()
# 'nan', 'Nan', '' (boşluk) olanları gerçek NaN yap
df_loan['title'] = df_loan['title'].replace({'': np.nan, 'nan': np.nan, 'Nan': np.nan})

def clean_term_guvenli(val):
    val = str(val)
    if '36' in val:
        return 36.0
    elif '60' in val:
        return 60.0
    else:
        return np.nan

df_loan['term'] = df_loan['term'].apply(clean_term_guvenli)

print("Term sütunu düzeltildi. Örnek değerler:")
print(df_loan['term'].unique())

# Faiz Temizliği
if df_loan['int_rate'].dtype == 'O':
    df_loan['int_rate'] = df_loan['int_rate'].str.replace('%', '').str.strip().astype(float)

# Tarih Formatı
df_loan['issue_d'] = pd.to_datetime(df_loan['issue_d'], format='%b-%Y', errors='coerce')

print(f"Orijinal Boyutlar -> Loan: {df_loan.shape}, Borrower: {df_borrower.shape}")

Term sütunu düzeltildi. Örnek değerler:
[36. 60. nan]
Orijinal Boyutlar -> Loan: (2260701, 18), Borrower: (2260701, 12)


In [ ]:
import re

df_states_ref = pd.read_csv('/content/us-state-codes.csv')
# 1. Metin Düzeltmeleri
df_borrower['emp_title'] = df_borrower['emp_title'].astype(str).str.strip().str.title()
df_borrower['emp_title'] = df_borrower['emp_title'].replace({'Nan': np.nan, 'Null': np.nan})

df_borrower['home_ownership'] = df_borrower['home_ownership'].astype(str).str.strip().str.title()
df_borrower['home_ownership'] = df_borrower['home_ownership'].replace({'NAN': np.nan, 'NULL': np.nan})

state_code_col = df_states_ref.columns[0] # 'state_code'
state_name_col = df_states_ref.columns[1] # 'state'
df_states_ref['clean_code'] = df_states_ref[state_code_col].astype(str).str.replace('US-', '', regex=False)

# Şimdi temiz kod ile eşleştirme sözlüğü yapıyoruz: {'AK': 'Alaska', 'AL': 'Alabama'}
state_map = dict(zip(df_states_ref['clean_code'], df_states_ref[state_name_col]))

# Eşleştirme
df_borrower['state_description'] = df_borrower['addr_state'].map(state_map)
df_borrower['state_description'] = df_borrower['state_description'].fillna(df_borrower['addr_state'])

# emp_length numeric
def clean_emp_length_guvenli(val):
    if pd.isna(val): return np.nan
    val = str(val).lower().strip()

    if '<' in val: # "< 1 year" durumu
        return 0.0

    # İçindeki ilk sayıyı bul 
    match = re.search(r'(\d+)', val)
    if match:
        return float(match.group(1))
    else:
        return np.nan

df_borrower['emp_length_numeric'] = df_borrower['emp_length'].apply(clean_emp_length_guvenli)

print("--- Çalışma Süresi (emp_length) Kontrolü ---")
print(df_borrower['emp_length_numeric'].value_counts().sort_index())
# Yıllık Gelir Boşluk Doldurma
median_inc = df_borrower['annual_inc'].median()
df_borrower['annual_inc'] = df_borrower['annual_inc'].fillna(median_inc)

# Joint Gelir Temizliği
if 'annual_inc_joint' in df_borrower.columns:
    df_borrower['annual_inc_joint'] = df_borrower['annual_inc_joint'].fillna(0)
if 'dti_joint' in df_borrower.columns:
    df_borrower['dti_joint'] = df_borrower['dti_joint'].fillna(0)

--- Çalışma Süresi (emp_length) Kontrolü ---
emp_length_numeric
0.0     189988
1.0     148403
2.0     203677
3.0     180753
4.0     136605
5.0     139698
6.0     102628
7.0      92695
8.0      91914
9.0      79395
10.0    748005
Name: count, dtype: int64


In [ ]:
df_loan.to_csv('loan_base_cleaned.csv', index=False)
df_borrower.to_csv('borrower_info_cleaned.csv', index=False)
print(f"Loan Cleaned: {df_loan.shape}")
print(f"Borrower Cleaned: {df_borrower.shape}")
files.download('loan_base_cleaned.csv')
files.download('borrower_info_cleaned.csv')

------------------------------
Temizlik Bitti! Yeni Dosyalar Oluşturuldu:
Loan Cleaned: (2260701, 18)
Borrower Cleaned: (2260701, 14)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>